# To Do
- Vectorise past data list info
- 

In [4]:
import pandas as pd
import datetime as datetime
import numpy as np
import matplotlib.pylab as plt
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

path = '../data_20161116/'
train = pd.read_csv(path+'train.csv', parse_dates = ['dt_prediction_date', 'dt_target_date', 'dt_flight_date'])
test = pd.read_csv(path+'test.csv', parse_dates = ['dt_prediction_date', 'dt_target_date', 'dt_flight_date'])

target_cols = ['num_pax_000_014_mins_before_sdt', 'num_pax_015_029_mins_before_sdt',
     'num_pax_030_044_mins_before_sdt', 'num_pax_045_059_mins_before_sdt',
     'num_pax_060_074_mins_before_sdt', 'num_pax_075_089_mins_before_sdt',
     'num_pax_090_104_mins_before_sdt', 'num_pax_105_119_mins_before_sdt',
     'num_pax_120_134_mins_before_sdt', 'num_pax_135_149_mins_before_sdt',
     'num_pax_150_164_mins_before_sdt', 'num_pax_165_179_mins_before_sdt',
     'num_pax_180_194_mins_before_sdt', 'num_pax_195_209_mins_before_sdt',
     'num_pax_210_224_mins_before_sdt', 'num_pax_225_239_mins_before_sdt',
     'num_pax_240plus_mins_before_sdt']


# Delete for model simplicity
del train["cat_i_airport"]
del train["cat_i_city"]

# We will predict on this sample
target_rows = train[train["cat_case_type"]=="Target"]
sample = target_rows.sample(n=int(len(target_rows)/100.0),random_state=42)
# Remove these boyos from the training set
rows = sample.index.values
train = train.drop(rows)

########################################################
# 3. Define functions which we will use later in the script
########################################################
def check_for_negatives_in_pred(df_pred, l_cols_to_range_over):
    '''A negative number of passengers turning up in a 15 minute window is not valid, so we set any negatives predictions to zero'''
    df_pred[df_pred[l_cols_to_range_over] < 0] = 0
    return df_pred

def calculate_score(df_target_cases, df_predictions):
    '''Root-mean-squared error is the chosen error metric. This function calculates and returns the root-mean-squared error'''
    f_rmse = np.sqrt(mean_squared_error(df_target_cases, df_predictions))
    return f_rmse

def func(x,df):
    """
    Predict on inputted row "x" using either past Expl data if available,
    or KNN based on permitted past data if not.
    """
    #    print(x["dt_prediction_date"])
    
    ## If there's allowed Expl. data for the same flight, just use the mean values
    expl_avail = df[df["id"] == x["id"]]
    if len(expl_avail) > 0:
        expl_avail = expl_avail[target_cols].mean(axis=0).astype(int)
        return expl_avail
        
    ## If no historical data do a KNN model...
    else:
        # filter by past dates
        permitted_data = df[df["dt_prediction_date"]<=x["dt_prediction_date"]]
        # filter by allowed model
        permitted_data = df[df["s_model_type"] == x["s_model_type"]]
        
        # now we can predict on the permitted data
        #temp_df = clean(df.copy())
        # Do some Cleaning
        
        
        ## Need to first attach X_train and X_pred and transform/delete/encode consistently
        Y_train = np.array(permitted_data[target_cols])
        X_train = permitted_data[[col for col in permitted_data.columns.values if col not in target_cols]]
        
        #Y_pred = x[target_cols]
        X_pred = x[[col for col in permitted_data.columns.values if col not in target_cols]]     
        
        
        ## Now transform
        temp_df = X_train.append(X_pred)
        
        categoricals = ['cat_sdt_hour',
                    #'cat_i_airport', 'cat_i_city',
                    'cat_destination_group_id', 'cat_longhaul_ind',
                   "cat_flight_class_type_id"]

        ordinals = [col for col in temp_df.columns.values if "ord" in col]

        dates = ['num_flight_year', 'num_flight_month', 'num_flight_weekofyear', 'num_flight_dayofweek']

        numericals = [feat for feat in temp_df.select_dtypes(include=["int","float"]).columns.values
                      if feat not in dates+ordinals+categoricals]

        ##########################
        # Change Cat Departure Time to 3 categories
        ##########################

        def time_binner(y):
            if y < 3 or y >= 19:
                return "Late"
            elif y >= 4 and y<= 11: 
                return "Day"
            else:
                return "Morning"

        temp_df["cat_sdt_hour"] = temp_df["cat_sdt_hour"].apply(time_binner)

        ##############
        # Perform one hot encoding on the categorical features
        ###############
        for cat in categoricals[:]:
            dummies = pd.get_dummies(temp_df[cat])
            dummies.columns = [str(col_name) + "_" + cat for col_name in dummies.columns.values]            
            temp_df = temp_df.drop(cat,axis=1)
            temp_df = temp_df.join(dummies)
        
        
        # kill these for model simplicity
        cols_to_delete = [
                          "id",
                          'dt_prediction_date',
                          'dt_target_date',
                          's_model_type',
                          'cat_case_type',
                          'cat_i_flightno',
                          'dt_flight_date',
                          'cat_s_plane_capacity',
                         ]

        temp_df = temp_df.drop(cols_to_delete,axis=1)
        
        columns_to_keep = list(temp_df.columns.values)
        
#         print(x[["id","s_model_type"]])
#         stop
        #print(x[["id","s_model_type"]])
        #print([col for col in x.index if col not in cols_to_delete])
#        x = x.loc[[col for col in x.index if col not in target_cols]]
#        x = x.loc[[col for col in x.index if col not in cols_to_delete]]
     
#         Y_train = np.array(temp_df[target_cols])
#         X_train = (temp_df[[col for col in temp_df.columns.values if col not in target_cols]])
# #         print(X_train.columns.values)
# #         print(x.index)
#         X_train = np.array(X_train)
    
        min_max_scaler = MinMaxScaler()
        #minmax_scale = min_max_scaler.fit(temp_df
#        print(columns_to_keep)
        temp_df[columns_to_keep] = min_max_scaler.fit_transform(temp_df[columns_to_keep])
        
        x = np.array(temp_df.iloc[-1])
        X_train = np.array(temp_df.iloc[:-1])
        
        
        neigh = KNeighborsRegressor(n_neighbors=3)
        neigh.fit(X_train, Y_train)
        
#         X_test = x
#         X_test = minmax_scale.transform(X_test)
#         print(X_test)
        
        Y_pred = neigh.predict(x)
       # print(len((Y_pred).flatten()))
       # print(df.columns.values)
        return pd.Series(Y_pred.flatten(),index=pd.Series(target_cols))
    
    #return np.array(permitted_data.index).flatten()

abc = sample
print(len(sample))
stop
prev = (np.array(abc[target_cols]))
abc[target_cols] = abc.apply(lambda row : func(row,train),axis=1)
print(abc)
now = (np.array(abc[target_cols]))
print(calculate_score(prev,now))
#print(sample["past indices"])

# for i, row in sample.iterrows():
#     print(row)
#     day = row["dt_prediction_date"]
#     past_data = train[train["dt_prediction_date"]<=day]
#     print(len(past_data))

119


NameError: name 'stop' is not defined